<a href="https://colab.research.google.com/github/SankethSingh/Text-Translation_BERT/blob/main/m_BERT_translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
! pip install transformers datasets torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [4]:
from transformers import BertTokenizer, BertForMaskedLM

# Load the m-BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertForMaskedLM.from_pretrained('bert-base-multilingual-cased')


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.

In [5]:
pip install datasets

In [6]:
from datasets import load_dataset

# Use 'wmt16' for Czech to English translation dataset
dataset = load_dataset('wmt16', 'cs-en')

# You can split it into train and validation/test sets if needed
train_dataset = dataset['train']  # Training data
test_dataset = dataset['test']    # Test data (optional)

print(train_dataset)
print(test_dataset)
train_dataset['translation']

README.md:   0%|          | 0.00/11.1k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/177M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/390k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/469k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/997240 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2656 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2999 [00:00<?, ? examples/s]

Dataset({
    features: ['translation'],
    num_rows: 997240
})
Dataset({
    features: ['translation'],
    num_rows: 2999
})


[{'cs': 'Následný postup na základě usnesení Parlamentu: viz zápis',
  'en': "Action taken on Parliament's resolutions: see Minutes"},
 {'cs': 'Předložení dokumentů: viz zápis',
  'en': 'Documents received: see Minutes'},
 {'cs': 'Písemná prohlášení (článek 116 jednacího řádu): viz zápis',
  'en': 'Written statements (Rule 116): see Minutes'},
 {'cs': 'Texty smluv dodané Radou: viz zápis',
  'en': 'Texts of agreements forwarded by the Council: see Minutes'},
 {'cs': 'Složení Parlamentu: viz zápis',
  'en': 'Membership of Parliament: see Minutes'},
 {'cs': 'Členství ve výborech a delegacích: viz zápis',
  'en': 'Membership of committees and delegations: see Minutes'},
 {'cs': 'Budoucí akce v oblasti patentů (předložené návrhy usnesení): viz zápis',
  'en': 'Future action in the field of patents (motions for resolutions tabled): see Minutes'},
 {'cs': 'Pořad jednání příštího zasedání: viz zápis',
  'en': 'Agenda for next sitting: see Minutes'},
 {'cs': 'Ukončení zasedání', 'en': 'Closure

In [7]:
from datasets import load_dataset
from sklearn.model_selection import train_test_split

# Load the wmt16 dataset for Czech-English
dataset = load_dataset('wmt16', 'cs-en')

# Select a subset of 10,000 samples from the training dataset
subset_size = 10000
train_data = dataset['train'].shuffle(seed=42).select(range(subset_size))

# Extract Czech and English sentences from the subset
cs_sentences = [item['cs'] for item in train_data['translation']]
en_sentences = [item['en'] for item in train_data['translation']]

# Split the 10,000 samples into training and test sets (80% train, 20% test)
cs_train, cs_test, en_train, en_test = train_test_split(
    cs_sentences, en_sentences, test_size=0.2, random_state=42
)

# Print some samples to verify
print(f"Training set size: {len(cs_train)}")
print(f"Test set size: {len(cs_test)}")
print(cs_train[:5])  # First 5 Czech sentences from the training set
print(en_train[:5])  # First 5 English sentences from the training set


Training set size: 8000
Test set size: 2000
['Chtěl bych vyzvat předsednictví, aby vytrvalo ve slibném úsilí, na něž se přistoupilo na jarních zasedáních Evropské rady v roce 2007 a 2008.', 'Výsledky této studie budou zohledněny při revizi stávající strategie Společenství v oblasti endokrinních disruptorů a studie poskytne podkladové informace pro provádění stávajících právních předpisů EU, čímž mám na mysli jak nařízení o přípravcích na ochranu rostlin, tak i nařízení REACH.', 'Pokud bude rezervace stornována do 40 dní před datem příjezdu, žádný poplatek nebude účtován.', 'Akreditovaní lobbisté se také musí řídit kodexem chování.', 'jménem skupiny GUE/NGL. - (CS) Vážený pane předsedající, vážená paní komisařko, dámy a pánové, moje kolegyně z výboru Ilda Figueiredo odvedla jako pokaždé výbornou práci.']
['I would urge the Presidency to hold onto the promising efforts that were accepted at the European Council in the spring of 2007 and 2008.', 'The results from this study will be taken 

In [8]:
cs_sentences_new = [sentence.lower() for sentence in cs_train]
en_sentences_new = [sentence.lower() for sentence in en_train]


In [9]:
import unicodedata

def remove_accents(text):
    return ''.join(c for c in unicodedata.normalize('NFD', text) if unicodedata.category(c) != 'Mn')

cs_sentences_clean = [remove_accents(sentence) for sentence in cs_sentences_new]
en_sentences_clean = [remove_accents(sentence) for sentence in en_sentences_new]

print(cs_sentences_clean[:5])
print(en_sentences_clean[:5])

['chtel bych vyzvat predsednictvi, aby vytrvalo ve slibnem usili, na nez se pristoupilo na jarnich zasedanich evropske rady v roce 2007 a 2008.', 'vysledky teto studie budou zohledneny pri revizi stavajici strategie spolecenstvi v oblasti endokrinnich disruptoru a studie poskytne podkladove informace pro provadeni stavajicich pravnich predpisu eu, cimz mam na mysli jak narizeni o pripravcich na ochranu rostlin, tak i narizeni reach.', 'pokud bude rezervace stornovana do 40 dni pred datem prijezdu, zadny poplatek nebude uctovan.', 'akreditovani lobbiste se take musi ridit kodexem chovani.', 'jmenem skupiny gue/ngl. - (cs) vazeny pane predsedajici, vazena pani komisarko, damy a panove, moje kolegyne z vyboru ilda figueiredo odvedla jako pokazde vybornou praci.']
['i would urge the presidency to hold onto the promising efforts that were accepted at the european council in the spring of 2007 and 2008.', 'the results from this study will be taken into account in reviewing the existing commu

In [10]:
!pip install transformers datasets

In [11]:
pip install stanza

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 39.5 MB/s eta 0:00:00


In [12]:
import spacy

# Load the English spaCy model
nlp_en = spacy.load("en_core_web_sm")

def lemmatize_en(sentences):
    lemmatized_sentences = []
    for sentence in sentences:
        doc = nlp_en(sentence)
        lemmatized_sentence = ' '.join([token.lemma_ for token in doc])
        lemmatized_sentences.append(lemmatized_sentence)
    return lemmatized_sentences

import stanza

# Load the Czech stanza model
stanza.download('cs')  # Run this once to download the model
nlp_cs = stanza.Pipeline('cs')

def lemmatize_cs(sentences):
    lemmatized_sentences = []
    for sentence in sentences:
        doc = nlp_cs(sentence)
        lemmatized_sentence = ' '.join([word.lemma for sent in doc.sentences for word in sent.words])
        lemmatized_sentences.append(lemmatized_sentence)
    return lemmatized_sentences
# Assuming cs_sentences and en_sentences are defined lists containing your sentences

# Lemmatize the English sentences
lemmatized_en_sentences = lemmatize_en(en_sentences_clean)

# Lemmatize the Czech sentences
lemmatized_cs_sentences = lemmatize_cs(cs_sentences_clean)

# Print the results
print("Lemmatized English Sentences:", lemmatized_en_sentences[:5])  # First 5 lemmatized English sentences
print("Lemmatized Czech Sentences:", lemmatized_cs_sentences[:5])  # First 5 lemmatized Czech sentences


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: cs (Czech) ...


INFO:stanza:Downloaded file to /root/stanza_resources/cs/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: cs (Czech):
| Processor | Package      |
----------------------------
| tokenize  | pdt          |
| mwt       | pdt          |
| pos       | pdt_nocharlm |
| lemma     | pdt_nocharlm |
| depparse  | pdt_nocharlm |

INFO:stanza:Using device: cuda
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Done loading processors!


Lemmatized English Sentences: ['I would urge the presidency to hold onto the promising effort that be accept at the european council in the spring of 2007 and 2008 .', 'the result from this study will be take into account in review the exist community strategy on endocrine disruptor and to provide input to the implementation of the exist eu legislation , by which I mean both the plant protection product regulation and also reach .', 'if cancel up to 40 day before date of arrival , no fee will be charge .', 'accredit lobbyist must also comply with a code of conduct .', ', on behalf of the gue / ngl group . - ( cs ) mr chairman , commissioner , lady and gentleman , my committee colleague ilda figueiredo have , as always , do an excellent job .']
Lemmatized Czech Sentences: ['chtel být vyzvat predsednictvo , aby být vytrvat v slibno usil , na nez se pristoupit na jarnich zasedanich evropský rada v rok 2007 a 2008 .', 'vysledek teto studie být zohlednený pre revize stavajik strategie spole

In [13]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")
# Tokenize with special tokens ([CLS] at start, [SEP] at end)
tokenized_cs = tokenizer(
    lemmatized_cs_sentences,
    padding=True,
    truncation=True,
    max_length=128,
    add_special_tokens=True,  # Adds [CLS] at start and [SEP] at end
    return_tensors="pt"
)
tokenized_en = tokenizer(
    lemmatized_en_sentences,
    padding=True,
    truncation=True,
    max_length=128,
    add_special_tokens=True,  # Adds [CLS] at start and [SEP] at end
    return_tensors="pt"
)

# Displaying the output
print("Czech Tokenized Output (IDs):", tokenized_cs['input_ids'])
print("English Tokenized Output (IDs):", tokenized_en['input_ids'])


Czech Tokenized Output (IDs): tensor([[  101, 18643, 15636,  ...,     0,     0,     0],
        [  101,   190, 12682,  ...,     0,     0,     0],
        [  101, 59749, 19632,  ...,     0,     0,     0],
        ...,
        [  101, 10132, 16924,  ...,     0,     0,     0],
        [  101, 46267, 11418,  ...,     0,     0,     0],
        [  101,   190, 31104,  ...,     0,     0,     0]])
English Tokenized Output (IDs): tensor([[  101,   146, 10894,  ...,     0,     0,     0],
        [  101, 10105, 14493,  ...,     0,     0,     0],
        [  101, 12277, 10944,  ...,     0,     0,     0],
        ...,
        [  101, 14234, 12807,  ...,     0,     0,     0],
        [  101, 13800,   117,  ...,     0,     0,     0],
        [  101, 10106, 10531,  ...,     0,     0,     0]])


In [14]:
# build model for training

In [15]:
from transformers import BertModel, Trainer, TrainingArguments, AdamW
import torch

# Define a translation model based on m-BERT, optionally wrapping with seq2seq components
class TranslationModel(torch.nn.Module):
    def __init__(self, model_name="bert-base-multilingual-cased"):
        super(TranslationModel, self).__init__()
        self.encoder = BertModel.from_pretrained(model_name)
        self.decoder = torch.nn.Linear(self.encoder.config.hidden_size, self.encoder.config.vocab_size)

    def forward(self, input_ids, attention_mask, labels=None):
        encoder_outputs = self.encoder(input_ids=input_ids, attention_mask=attention_mask)
        logits = self.decoder(encoder_outputs.last_hidden_state)

        # Calculate loss if labels are provided (during training)
        loss = None
        if labels is not None:
            loss_fn = torch.nn.CrossEntropyLoss()
            # Flatten the tensors for cross-entropy calculation
            loss = loss_fn(logits.view(-1, self.encoder.config.vocab_size), labels.view(-1))

        return {"loss": loss, "logits": logits}

# Initialize model
model = TranslationModel()

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01
)

# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_cs,    ## train and val dataset setting
    eval_dataset=tokenized_val_dataset,
    tokenizer=tokenizer,
    optimizers=(AdamW(model.parameters(), lr=5e-5), None)  # optimizer and scheduler
)

# Train the model
trainer.train()

# Evaluate on the test set
results = trainer.evaluate(tokenized_test_dataset)
print("Evaluation Results:", results)

#ss

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


NameError: name 'tokenized_val_dataset' is not defined

In [ ]:
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertModel, AutoModel

class TranslationModel(nn.Module):
    def __init__(self, encoder_name, decoder_name, hidden_size=768):
        super(TranslationModel, self).__init__()

        # Load encoder and decoder models
        self.encoder = BertModel.from_pretrained(encoder_name)
        self.decoder = AutoModel.from_pretrained(decoder_name)

        # Define a linear layer to project encoder outputs to decoder's input size
        self.encoder_to_decoder = nn.Linear(hidden_size, hidden_size)

        # Define a linear layer for the final output (vocab size of Czech)
        self.output_layer = nn.Linear(hidden_size, self.decoder.config.vocab_size)

    def forward(self, input_ids, attention_mask, decoder_input_ids):
        # Encoder forward pass
        encoder_outputs = self.encoder(input_ids=input_ids, attention_mask=attention_mask)

        # Project encoder output to the decoder's input size
        encoder_hidden_states = self.encoder_to_decoder(encoder_outputs.last_hidden_state)

        # Decoder forward pass with encoder hidden states as additional input
        decoder_outputs = self.decoder(
            input_ids=decoder_input_ids,
            encoder_hidden_states=encoder_hidden_states,
            encoder_attention_mask=attention_mask
        )

        # Final output projection to Czech vocabulary
        logits = self.output_layer(decoder_outputs.last_hidden_state)

        return logits


In [ ]:
# Initialize the translation model
model = TranslationModel(encoder_name="bert-base-uncased", decoder_name="Helsinki-NLP/opus-mt-en-cs")
model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss(ignore_index=czech_tokenizer.pad_token_id)
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

# Forward pass and compute loss
model.train()
optimizer.zero_grad()
logits = model(input_ids=input_ids.to(device), attention_mask=attention_mask.to(device), decoder_input_ids=decoder_input_ids.to(device))

# Shift target tokens for the next word prediction
shifted_decoder_input_ids = decoder_input_ids[:, 1:].contiguous()
shifted_logits = logits[:, :-1].contiguous()

# Calculate loss
loss = criterion(shifted_logits.view(-1, shifted_logits.size(-1)), shifted_decoder_input_ids.view(-1))
loss.backward()
optimizer.step()

print(f"Training loss: {loss.item()}")


In [ ]:
model.eval()
with torch.no_grad():
    # Use the encoder output and generate tokens one by one
    decoder_input_ids = torch.tensor([[czech_tokenizer.bos_token_id]]).to(device)
    for _ in range(50):  # Limiting to max 50 tokens for translation output
        logits = model(input_ids=input_ids.to(device), attention_mask=attention_mask.to(device), decoder_input_ids=decoder_input_ids)
        next_token = logits[:, -1, :].argmax(dim=-1)
        decoder_input_ids = torch.cat([decoder_input_ids, next_token.unsqueeze(-1)], dim=-1)

        # Stop if the model generates an end-of-sequence token
        if next_token == czech_tokenizer.eos_token_id:
            break

    # Decode the generated sequence
    translation = czech_tokenizer.decode(decoder_input_ids[0], skip_special_tokens=True)
    print("Generated Translation:", translation)
